In [ ]:
from mne_bids_pipeline._config_import import _import_config
import mne
import numpy as np
from mne.time_frequency import tfr_morlet
cfg = _import_config(
        config_path='./mne-bids/config/mne-bids-pipeline.py'
    )
%matplotlib inline

In [ ]:

power = mne.time_frequency.read_tfrs('./data/ds003702/derivatives/mne-bids-pipeline/sub-01/eeg/sub-01_task-SocialMemoryCuing_power+sticks+tfr.h5', condition=0) # TODO: should probably get the path from the config too
power.apply_baseline(
    baseline=cfg.time_frequency_baseline,
    mode=cfg.time_frequency_baseline_mode,
)
if cfg.time_frequency_crop:
    power.crop(**cfg.time_frequency_crop)
%matplotlib inline

In [ ]:
fig_power = power.plot_topomap(ch_type='eeg', show=True)
fig_power.axes[0].set_title('Alpha: Stick')
fig_power.set_size_inches(5, 5)

In [ ]:
fig_power = power.plot_topo(title='Alpha: Stick', fig_facecolor="w", font_color="k", border="k", show=True)
fig_power.set_size_inches(9, 6)

In [ ]:
average_path = './data/ds003702/derivatives/mne-bids-pipeline/sub-average/eeg/sub-average_task-SocialMemoryCuing_ave.fif'
evokeds = {}
evokeds['avatar'] = mne.read_evokeds(average_path, condition='Grand average: avatar')
evokeds['sticks'] = mne.read_evokeds(average_path, condition='Grand average: sticks')
evokeds['contrast'] = mne.read_evokeds(average_path, condition='Grand average: avatar - sticks')


In [ ]:
freqs = np.arange(2.0, 30.0, 1)  # define frequencies of interest
n_cycles = freqs / freqs[0]
condition_names = ['avatar', 'sticks']
for condition_name in condition_names:
    condition = evokeds[condition_name]
    this_tfr = tfr_morlet(
        condition,
        freqs,
        n_cycles= n_cycles,
        return_itc=False,
    )
    this_tfr.apply_baseline(mode="logratio", baseline=(-1, 0))
    this_tfr.plot(combine = 'mean', show=True, tmin = 2.5, tmax = 3.5, title=condition_name)